In [ ]:
"""
Теперь решаем задачу регрессии - предскажем цены на недвижимость. 
Использовать датасет www.kaggle.com...iques/data (train.csv)
Данных немного, поэтому необходимо использовать 10-fold кросс-валидацию для оценки качества моделей
Построить случайный лес, вывести важность признаков
Обучить стекинг как минимум 3х моделей, использовать хотя бы 1 линейную модель и 1 нелинейную
Для валидации модели 2-го уровня использовать отдельный hold-out датасет, как на занятии
Показать, что использование ансамблей моделей действительно улучшает качество (стекинг vs другие модели сравнивать на hold-out)
В качестве решения: Jupyter notebook с кодом, комментариями и графиками
"""

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/Users/dubrov_arseniy/Desktop/Нетология/Data Scientist/ML/9. Outlier/train.csv')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
#Посмотрим, сколько всего строк и колонок
df.shape

(1460, 81)

In [4]:
# теперь посмотрим, есть ли пропуски 
null_check = pd.DataFrame({'type': df.dtypes,
                  'null': df.isnull().sum(), 
                '% of total': round(df.isnull().sum() / df["Id"].count(),2)*100,

                 
                 })
null_check.loc[null_check['null'] != 0]

,type,null,% of total
LotFrontage,float64,259,18.0
Alley,object,1369,94.0
MasVnrType,object,8,1.0
MasVnrArea,float64,8,1.0
BsmtQual,object,37,3.0
BsmtCond,object,37,3.0
BsmtExposure,object,38,3.0
BsmtFinType1,object,37,3.0
BsmtFinType2,object,38,3.0
Electrical,object,1,0.0


In [5]:
# Переменные Alley,FireplaceQu,MiscFeature, Fence, PoolQC имеют очень много пропусков, поэтому просто удалим их из датасета

df.drop(columns= ["Alley","FireplaceQu","MiscFeature", "Fence", "PoolQC"], inplace = True)

In [6]:
#В остальных случаях заменим пропуски на медиану (для формата float)
df.fillna(df.median(), inplace = True)
df.isnull().sum()

Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
                ..
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
SalePrice        0
Length: 76, dtype: int64

In [7]:
#Заполним пропуски значением нан для типа данных object
category = list(df.dtypes[df.dtypes == object].index)
df[category] = df[category].fillna('nan')

In [8]:
#Делаем категориальные переменнные с помощью onehot encoding
new_data = pd.get_dummies(df, columns = category)

In [9]:
# Объявим переменные Х и у
X = new_data[new_data.columns]
X.drop(columns = ['SalePrice'], inplace = True)
y = new_data['SalePrice']

In [41]:

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier 
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import RidgeCV

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [36]:
# Стандартизируем данные через pipline
work_model = make_pipeline(StandardScaler(), RandomForestClassifier())

In [38]:
model = RandomForestClassifier(
    n_estimators=50, 
    max_depth = 2
)
work_model.fit(X_train, y_train)
work_model.score(X_test, y_test)



0.0136986301369863

In [39]:
#Посмотрим точность предсказания на тренировочных и тестовых(боевых) данных
print(f'train: {round(work_model.score(X_train, y_train),5)}')
print(f'test: {round(work_model.score(X_test, y_test),5)}')

train: 1.0
test: 0.0137


##  Мы видим, что модель переобучилась и на тестовых данных не выдает результат

In [42]:
# Создаем стэккинг и обучаем его на наших данных 
# добавляем 3 базовых моделей и одну мета-модель 

Regressor = StackingRegressor(
    [
        ('LinearRegression', LinearRegression()),
        ('KNeighborsRegressor', KNeighborsRegressor()),
        ('DecisionTree', DecisionTreeRegressor())
    ], RidgeCV())

In [43]:
Regressor.fit(X_train, y_train)

StackingRegressor(estimators=[('LinearRegression', LinearRegression()),
                              ('KNeighborsRegressor', KNeighborsRegressor()),
                              ('DecisionTree', DecisionTreeRegressor())],
                  final_estimator=RidgeCV(alphas=array([ 0.1,  1. , 10. ])))

In [45]:
print(f'Score on train data {Regressor.score(X_train, y_train)}')
print(f'Score on test data {Regressor.score(X_test, y_test)}')
# посмотрим качество на трейне и тесте в тотале


for i in Regressor.named_estimators:# тут посмотрим по оценку по каждой нашей моделе
    print(f'Score on train data with model {i} {Regressor.named_estimators_[i].score(X_train, y_train)}')
    print(f'Score on test data with model {i} {Regressor.named_estimators_[i].score(X_test, y_test)}')

Score on train data 0.9223860973231867
Score on test data 0.8039215124710178
Score on train data with model LinearRegression 0.9342629611882138
Score on test data with model LinearRegression 0.8887424946608482
Score on train data with model KNeighborsRegressor 0.7379405825437533
Score on test data with model KNeighborsRegressor 0.6781510927892045
Score on train data with model DecisionTree 1.0
Score on test data with model DecisionTree 0.7824897823222284


## Мы видим, что стекинг показывает результат. Модель переобучается только на дереве. Лучше всего показывает себя линейная регрессия